# PyQGIS

**Come usarlo**  
- Le celle di codice vanno copiate/incollate nella **Console Python di QGIS** (o trasformate in *Processing Script*).  

**Obiettivi**  
1. Fondamentali PyQGIS (progetto, layer, CRS, salvataggi).  
2. Processing da Python (buffer/intersect/aggregate).  
3. Automazione (funzioni riusabili).  
4. Export cartografico (layout da codice).

**Dataset reali**: ISTAT *Comuni 2025*, OSM Geofabrik *Nord-Est*, GTFS *Trentino Trasporti*.

**Cosa tocchiamo in PyQGIS**  
- `QgsProject`, `QgsVectorLayer`, `QgsExpression`, `QgsFeatureRequest`  
- Processing (`processing.run`)  
- `QgsVectorFileWriter` per scrivere **GeoPackage**  
- `QgsPrintLayout` per **layout da script**

**Regola d'oro**: calcoli in **metri** → **EPSG:25832**.

## 1) Setup progetto e cartelle 

### Da `pathlib`
* `Path`: Per creare e manipolare percorsi di file e cartelle in modo indipendente dal sistema operativo.
* `Path.home()`: Ottiene la cartella "home" dell'utente.
* `Path.mkdir()`: Crea una cartella.
* `Path.as_posix()`: Converte il percorso in una stringa di testo usando lo slash `/` come separatore (preferito da QGIS).

### Da `qgis.core`
* `QgsProject.instance()`: Ottiene un riferimento al progetto QGIS attualmente aperto.
* `QgsProject.setCrs()`: Imposta il Sistema di Riferimento delle Coordinate (CRS) per il progetto.
* `QgsCoordinateReferenceSystem.fromEpsgId()`: Crea un oggetto CRS partendo dal suo codice EPSG (es. 25832).

In [ ]:
# Importa le librerie necessarie (pathlib per i percorsi, qgis.core per le funzioni QGIS)
from pathlib import Path
from qgis.core import QgsProject, QgsCoordinateReferenceSystem

# Definisce la cartella di base per i dati nella home dell'utente
BASE_DIR = Path.home()/ "Corso_PyQGIS" / "data_lezione_04" / "qgis_data"
# Crea la cartella se non esiste già
BASE_DIR.mkdir(parents=True, exist_ok=True)

# Definisce il percorso completo del file GeoPackage 
GPKG_OUT = (BASE_DIR / "trento.gpkg").as_posix()

# Imposta il Sistema di Riferimento (CRS) del progetto QGIS su EPSG:25832 (UTM 32N)
QgsProject.instance().setCrs(QgsCoordinateReferenceSystem.fromEpsgId(25832))

# Stampa i percorsi definiti per verificaAC
print("Cartella dati:", BASE_DIR)
print("GeoPackage:", GPKG_OUT)

## 2) Funzioni di utilità 

### Download ed Estrazione (`dload`, `unzip`)

Queste sono funzioni "helper" per automatizzare il recupero dei dati.
* `dload`: Scarica un file da un URL.
* `unzip`: Estrae l'archivio.
* **Scopo**: Rendere lo script auto-sufficiente, senza passaggi manuali di preparazione dei dati.

###  Caricamento Shapefile (`find_shp`, `load_shp`)

Risolvono un problema comune: i file negli zip spesso hanno nomi imprevedibili (es. `comm_tn_2020.shp`).

* `find_shp`: Cerca ricorsivamente nella cartella estratta un file `.shp` che contenga nel nome una `keyword` chiave (es. "comuni").
* `load_shp`: Carica il file trovato da `find_shp` come `QgsVectorLayer` e controlla che sia valido (`.isValid()`).


###  Riproiezione (`to_epsg`)

Questa è una funzione cruciale per la **standardizzazione dei dati**.

* Prima controlla se il layer è **già** nel CRS corretto (definito in `TARGET_EPSG`). Se sì, non fa nulla per efficienza.
* Altrimenti, usa l'algoritmo di processing `native:reprojectlayer`.
* L'opzione `'OUTPUT': 'memory:'` è la più importante: crea il layer riproiettato **solo in RAM**, rendendo l'operazione molto più veloce rispetto alla scrittura di un file temporaneo.


### Scrittura "Robusta" (`save_gpkg`)

Questa funzione per salvare il GeoPackage è "**robusta**" perché gestisce due aspetti critici che spesso causano errori:

1.  **Gestione della Sovrascrittura**: Il codice controlla se il file GPKG esiste già.
    * Se **sì**, usa `CreateOrOverwriteLayer`. Questo **aggiorna/sovrascrive solo quel layer** dentro il file, lasciando intatti eventuali altri layer già presenti.
    * Se **no**, crea il file da zero.

2.  **Controllo Reale degli Errori**: La scrittura di file in QGIS (`QgsVectorFileWriter`) **non lancia un'eccezione Python** se fallisce, ma restituisce un codice di stato.
    * La funzione helper `_unpack_writer_result` interpreta questo codice.
    * Lo script controlla attivamente se `err_code != QgsVectorFileWriter.NoError` e, in caso di fallimento, lancia un'eccezione chiara. Questo evita "**fallimenti silenziosi**" (quando lo script termina senza errori, ma il file non è stato scritto).

In [ ]:
import os, zipfile, tempfile, shutil
from urllib.request import urlopen
from pathlib import Path

# Importa le classi necessarie da QGIS e l'API di processing
from qgis.core import (
    QgsVectorLayer, QgsVectorFileWriter, QgsCoordinateTransformContext,
    QgsCoordinateReferenceSystem, QgsExpression, QgsFeatureRequest
)
import processing as pr

# Definisce l'EPSG target per la riproiezione (es. ETRS89 / UTM zona 32N)
TARGET_EPSG = 25832

# Funzione per scaricare un file da un URL in una cartella specifica
def dload(url: str, dst_folder: str | Path) -> Path:
    # Estrae il nome del file dall'URL
    fn = os.path.basename(url.split("?")[0]) or "download.zip"
    out = Path(dst_folder) / fn
    # Scarica il file e lo salva sul disco
    with urlopen(url) as r, open(out, "wb") as f:
        shutil.copyfileobj(r, f)
    return out

# Funzione per estrarre un file .zip in una cartella di destinazione
def unzip(zip_path: str | Path, dst_folder: str | Path) -> Path:
    dst_folder = Path(dst_folder)
    with zipfile.ZipFile(zip_path, 'r') as z: z.extractall(dst_folder)
    return dst_folder

# Funzione per cercare uno shapefile (.shp) in una cartella (ricorsivamente) 
def find_shp(base: str | Path, keywords: list[str]) -> Path:
    base = Path(base)
    keys = [k.lower() for k in keywords]
    # Cerca ricorsivamente (rglob) tutti i file .shp
    for shp in base.rglob("*.shp"):
        # Controlla se una keyword è presente nel nome del file
        if any(k in shp.name.lower() for k in keys):
            return shp
    raise FileNotFoundError(f"Nessuno shapefile con {keywords} in {base}")

# Funzione per caricare uno shapefile come QgsVectorLayer in QGIS
def load_shp(path: str | Path, name: str) -> QgsVectorLayer:
    vl = QgsVectorLayer(str(path), name, "ogr")
    if not vl.isValid():
        raise Exception(f"Layer non valido: {path}")
    return vl

# Funzione per riproiettare un layer nel CRS target (TARGET_EPSG) se non lo è già
def to_epsg(layer: QgsVectorLayer, epsg: int = TARGET_EPSG) -> QgsVectorLayer:
    # Se il CRS è già corretto, restituisce il layer originale
    if layer.crs().authid() == f"EPSG:{epsg}":
        return layer
    # Altrimenti, usa l'algoritmo di processing "reprojectlayer"
    return pr.run("native:reprojectlayer", {
        'INPUT': layer,
        'TARGET_CRS': QgsCoordinateReferenceSystem.fromEpsgId(epsg),
        'OUTPUT': 'memory:' # Crea il layer riproiettato in memoria
    })['OUTPUT']

# Funzione "helper" interna per interpretare il codice di errore dei metodi di scrittura QGIS
def _unpack_writer_result(res):
    err_code, err_msg = None, ""
    # Gestisce diversi formati di ritorno (int, tuple, oggetto)
    if isinstance(res, int):
        err_code = res
    elif isinstance(res, (tuple, list)):
        err_code = res[0] if len(res) > 0 else None
        if len(res) > 1 and isinstance(res[1], str):
            err_msg = res[1]
    else:
        err_code = getattr(res, "status", None)
        err_msg = getattr(res, "errorMessage", "")
    return err_code, err_msg

# Funzione per salvare un QgsVectorLayer in un file GeoPackage (.gpkg)
def save_gpkg(layer, gpkg_path: str | Path, layer_name: str) -> None:
    gpkg_path = Path(gpkg_path)
    opts = QgsVectorFileWriter.SaveVectorOptions()
    opts.driverName = "GPKG"
    opts.layerName = layer_name # Imposta il nome del layer *dentro* il GeoPackage
    
    # Gestisce la sovrascrittura
    if gpkg_path.exists():
        # Se il file GPKG esiste, crea o sovrascrive il LAYER al suo interno
        opts.actionOnExistingFile = QgsVectorFileWriter.CreateOrOverwriteLayer
    else:
        # Se il file non esiste, lo crea
        opts.actionOnExistingFile = QgsVectorFileWriter.CreateOrOverwriteFile
    
    # Esegue la scrittura del file vettoriale
    res = QgsVectorFileWriter.writeAsVectorFormatV3(
        layer, gpkg_path.as_posix(), QgsCoordinateTransformContext(), opts
    )
    
    # Controlla il risultato della scrittura
    err_code, err_msg = _unpack_writer_result(res)
    if err_code != QgsVectorFileWriter.NoError:
        raise Exception(f"Scrittura GPKG fallita per '{layer_name}': {err_msg}")

## 3) Download e preparazione dati 

Questo script esegue le stesse operazioni di base (download ISTAT, OSM, GTFS), ma utilizza una serie di **tecniche avanzate per ottimizzare le prestazioni**, specialmente quando si lavora con file di grandi dimensioni (come i dati OSM di Geofabrik).

L'obiettivo è **ridurre i tempi di attesa** e l'**uso del disco** evitando di decomprimere inutilmente file ZIP da centinaia di MB o GB.


### 1. Downloader " (`dload`)
* La funzione `dload` è stata scritta per usare `QgsNetworkAccessManager` (il gestore di rete di QGIS) e un `QEventLoop`.
* **Perché?**: Il download di file di grandi dimensioni può bloccare l'interfaccia utente di QGIS, facendola sembrare "congelata". Questo approccio **mantiene la GUI reattiva** scaricando i dati in background (in "chunk") e attendendo attivamente il completamento (`loop.exec()`) senza bloccare il thread principale.


### 2. Lettura Diretta da ZIP (`/vsizip/`)
Questa è l'ottimizzazione più importante per ISTAT e OSM. Invece di estrarre l'intero archivio ZIP su disco, usiamo un "filesystem virtuale" di GDAL/OGR.

* **`find_shp_in_zip(zip_path, ...)`**: Questa funzione gurda dentro l'archivio (`z.namelist()`) e trova il *percorso interno* dello shapefile che ci serve (es. `Limiti01012025/Com01012025/Com01012025_WGS84.shp`), il tutto **senza estrarre nulla**.
* **`vsizip_layer(zip_path, inner, ...)`**: Questa funzione crea un `QgsVectorLayer` usando un percorso speciale:
  `"/vsizip/percorso/archivio.zip/percorso/interno/file.shp"`
* **Perché?**: QGIS/GDAL legge i dati *direttamente* dallo ZIP on-the-fly. Questo fa risparmiare minuti di tempo e gigabyte di spazio su disco, specialmente con il file OSM.


### 3. "Pre-Clip" con Bounding Box (OSM)
Eseguire un clip poligonale (comune di Trento) sull'intero layer OSM del Nord-Est Italia è **estremamente lento**, anche con `/vsizip/`. Applichiamo una strategia in due passaggi:

1.  **`extent_in_epsg(trento, 4326)`**: Calcoliamo l'estensione (Bounding Box, BBOX) del comune di Trento e la trasformiamo nel CRS dei dati OSM (WGS84 / EPSG:4326).
2.  **`native:extractbyextent`**: Usiamo questo algoritmo (molto veloce) per fare un primo "ritaglio" rettangolare (il BBOX) sul layer `/vsizip/`. Questo scarta il 99% dei dati (es. tutto ciò che è in Veneto ed Emilia-Romagna) in pochi secondi.
3.  **`native:reprojectlayer`**: Riproiettiamo *solo* il risultato del BBOX clip (dati molto più piccoli).
4.  **`native:clip`**: Eseguiamo il clip poligonale finale (preciso) sul poligono di Trento. Questa operazione è ora velocissima perché lavora solo su un'area leggermente più grande di Trento.


### 4. Estrazione Selettiva da ZIP (GTFS)
Anche i file GTFS sono archivi ZIP. Non abbiamo bisogno di tutti i file (es. `trips.txt`, `calendar.txt`), ma **solo di `stops.txt`**.

* **`with zf.open(member) ...`**: Invece di usare `zipfile.extractall()`, troviamo il membro `stops.txt` e usiamo `zf.open()` per estrarre *solo ed esclusivamente quel file*, risparmiando tempo e operazioni I/O su disco.

### 5. Disattivazione del Rendering
* **`iface.mapCanvas().setRenderFlag(False)`**: Prima di iniziare l'elaborazione pesante, diciamo a QGIS di **smettere di aggiornare la mappa**.
* **Perché?**: Quando si creano molti layer in memoria (`"OUTPUT": "memory:"`), l'interfaccia di QGIS potrebbe cercare di disegnarli, causando rallentamenti e "lag". Disattivando il rendering, l'intero script viene eseguito molto più fluidamente. Viene riattivato alla fine.

In [ ]:
from pathlib import Path
import os, tempfile, zipfile

from qgis.PyQt.QtCore import QEventLoop, QUrl
from qgis.PyQt.QtNetwork import QNetworkRequest
from qgis.core import (
    QgsNetworkAccessManager, QgsMessageLog, Qgis,
    QgsExpression, QgsFeatureRequest, QgsProject, QgsVectorLayer,
    QgsCoordinateReferenceSystem, QgsCoordinateTransform
)
import processing as pr

def dload(url: str, dst_folder: str | Path) -> Path:
    """Scarica un file in dst_folder mantenendo la GUI reattiva."""
    fn = os.path.basename(url.split("?")[0]) or "download.zip"
    out = Path(dst_folder) / fn

    # se già presente e non vuoto, riusa
    if out.exists() and out.stat().st_size > 0:
        return out

    nam = QgsNetworkAccessManager.instance()
    req = QNetworkRequest(QUrl(url))
    reply = nam.get(req)

    loop = QEventLoop()
    f = open(out, "wb")

    def on_ready_read():
        data = reply.readAll()
        if data:
            f.write(bytes(data))

    def on_finished():
        try:
            data = reply.readAll()
            if data:
                f.write(bytes(data))
        finally:
            f.close()
        reply.deleteLater()
        loop.quit()

    reply.readyRead.connect(on_ready_read)
    reply.finished.connect(on_finished)

    # Qt6/Qt5
    try: loop.exec()
    except AttributeError: loop.exec_()

    if reply.error():
        err = reply.errorString()
        QgsMessageLog.logMessage(f"Errore download {fn}: {err}", "PyQGIS", Qgis.Critical)
        raise RuntimeError(f"Download fallito: {err}")

    return out

#Helper leggeri per evitare estrazioni complete 
def find_shp_in_zip(zip_path: Path, keywords: list[str]) -> str:
    """Ritorna il percorso interno di uno .shp nel .zip che contiene una delle keywords."""
    kws = [k.lower() for k in keywords]
    with zipfile.ZipFile(zip_path, 'r') as z:
        cands = [n for n in z.namelist()
                 if n.lower().endswith(".shp") and any(k in os.path.basename(n).lower() for k in kws)]
    if not cands:
        raise FileNotFoundError(f"Nessuno shapefile con {keywords} in {zip_path.name}")
    # preferisci il primo nella root dell'archivio
    cands.sort(key=lambda n: n.count("/"))
    return cands[0]

def vsizip_layer(zip_path: Path, inner: str, name: str) -> QgsVectorLayer:
    """Crea un QgsVectorLayer che legge direttamente dentro lo zip (niente unzip)."""
    p = f"/vsizip/{zip_path.as_posix()}/{inner}"
    lyr = QgsVectorLayer(p, name, "ogr")
    if not lyr.isValid():
        raise RuntimeError(f"Layer non valido: {p}")
    return lyr

def extent_in_epsg(layer: QgsVectorLayer, epsg: int):
    """Trasforma l'extent di 'layer' nell'EPSG richiesto (per bbox clipping)."""
    src = layer.crs()
    dst = QgsCoordinateReferenceSystem.fromEpsgId(epsg)
    ct = QgsCoordinateTransform(src, dst, QgsProject.instance())
    return ct.transformBoundingBox(layer.extent())

#URL sorgenti
ISTAT_URL = "https://www.istat.it/storage/cartografia/confini_amministrativi/non_generalizzati/2025/Limiti01012025.zip"
# OSM Nord-Est (grande) — se vuoi dataset più piccolo, prova a sostituire con quello regionale:
GEOFABRIK_URL = "https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip"
# Esempio alternativo più leggero (se esiste/valido): "https://download.geofabrik.de/europe/italy/trentino-alto-adige-latest-free.shp.zip"
GTFS_URLS = [
    "https://www.trentinotrasporti.it/opendata/google_transit_urbano_tte.zip",
    "https://www.trentinotrasporti.it/opendata/google_transit_extraurbano_tte.zip",
]

#Dir temporanea e switch rendering 
tmp = tempfile.mkdtemp(prefix="pyqgis_trento_nb_opt_")
print("Temp:", tmp)

# Disattiva rendering durante i passi pesanti (riduce lag/blocchi)
_render_flag_reset = None
try:
    _render_flag_reset = iface.mapCanvas().renderFlag()
    iface.mapCanvas().setRenderFlag(False)
except Exception:
    _render_flag_reset = None  # se non in GUI, ignora

#ISTAT: leggere direttamente dentro lo ZIP, filtrare Trento, salvare 
print("Processo ISTAT...")
istat_zip = dload(ISTAT_URL, tmp)
istat_shp_in = find_shp_in_zip(Path(istat_zip), ["com", "comuni"])
istat_lyr = vsizip_layer(Path(istat_zip), istat_shp_in, "istat_comuni")

# Filtra Trento (per codice, altrimenti per nome)
fields = [f.name() for f in istat_lyr.fields()]
key = next((c for c in ["PRO_COM_T","PRO_COM","COD_ISTAT","ISTAT","COD_PRO_COM"] if c in fields), None)
if key:
    expr = QgsExpression(f'"{key}" = \'022205\'')
else:
    nm = next((c for c in ["COMUNE","DEN_UTS","DENOMINAZ","DEN_COM","DENOM"] if c in fields), None)
    if not nm: raise Exception(f"Campi ISTAT inattesi: {fields}")
    expr = QgsExpression(f'"{nm}" ILIKE \'Trento\'')

trento_sel = istat_lyr.materialize(QgsFeatureRequest(expr))

# Reproject to 25832

from qgis.core import QgsCoordinateTransformContext
trento_25832 = pr.run("native:reprojectlayer", {
    "INPUT": trento_sel, "TARGET_CRS": "EPSG:25832", "OPERATION": "", "OUTPUT": "memory:"
})["OUTPUT"]

# Salva subito il poligono base
save_gpkg(trento_25832, GPKG_OUT, "comune_trento")

#OSM: NIENTE unzip, pre-clip per bbox su ZIP, poi clip poligonale 
print("Processo OSM...")
osm_zip = dload(GEOFABRIK_URL, tmp)
roads_shp_in = find_shp_in_zip(Path(osm_zip), ["roads"])
blds_shp_in  = find_shp_in_zip(Path(osm_zip), ["buildings"])

roads_zip_lyr = vsizip_layer(Path(osm_zip), roads_shp_in, "osm_roads_wgs84")
blds_zip_lyr  = vsizip_layer(Path(osm_zip), blds_shp_in,  "osm_buildings_wgs84")

# bounding box di Trento in WGS84 (OSM shapefile è tipicamente EPSG:4326)
roi_wgs84 = extent_in_epsg(trento_25832, 4326)

# 1) Pre-taglio per BBOX (molto più veloce su dataset grandi)
roads_bbox = pr.run("native:extractbyextent", {
    "INPUT": roads_zip_lyr,
    "EXTENT": f"{roi_wgs84.xMinimum()},{roi_wgs84.xMaximum()},{roi_wgs84.yMinimum()},{roi_wgs84.yMaximum()}",
    "CLIP": True, "OUTPUT": "memory:"
})["OUTPUT"]

blds_bbox = pr.run("native:extractbyextent", {
    "INPUT": blds_zip_lyr,
    "EXTENT": f"{roi_wgs84.xMinimum()},{roi_wgs84.xMaximum()},{roi_wgs84.yMinimum()},{roi_wgs84.yMaximum()}",
    "CLIP": True, "OUTPUT": "memory:"
})["OUTPUT"]

# 2) Riproietta i pre-tagli a 25832
roads_25832 = pr.run("native:reprojectlayer", {
    "INPUT": roads_bbox, "TARGET_CRS": "EPSG:25832", "OPERATION": "", "OUTPUT": "memory:"
})["OUTPUT"]

blds_25832 = pr.run("native:reprojectlayer", {
    "INPUT": blds_bbox, "TARGET_CRS": "EPSG:25832", "OPERATION": "", "OUTPUT": "memory:"
})["OUTPUT"]

# 3)  Indici spaziali prima del clip poligonale
pr.run("native:createspatialindex", {"INPUT": roads_25832})
pr.run("native:createspatialindex", {"INPUT": blds_25832})

# 4) Clip finale sul poligono di Trento
clip_roads = pr.run("native:clip", {
    "INPUT": roads_25832, "OVERLAY": trento_25832, "OUTPUT": "memory:"
})["OUTPUT"]

clip_blds = pr.run("native:clip", {
    "INPUT": blds_25832,  "OVERLAY": trento_25832, "OUTPUT": "memory:"
})["OUTPUT"]

# Salvataggi
save_gpkg(clip_roads, GPKG_OUT, "strade")
save_gpkg(clip_blds,  GPKG_OUT, "edifici")

#GTFS: estrai SOLO stops.txt da ZIP (evita estrazioni complete) 
print("Processo GTFS...")
stop_layers = []
for url in GTFS_URLS:
    zpath = dload(url, tmp)
    with zipfile.ZipFile(zpath, 'r') as zf:
        # trova stops.txt ovunque sia dentro lo zip
        member = next((n for n in zf.namelist() if n.lower().endswith("/stops.txt") or n.lower() == "stops.txt"), None)
        if not member:
            print("ATTENZIONE: stops.txt non trovato in", zpath)
            continue
        out_csv = Path(tmp) / ("stops_" + Path(url).stem + ".txt")
        with zf.open(member) as src, open(out_csv, "wb") as dst:
            dst.write(src.read())

    uri = f'file:///{out_csv.as_posix()}?delimiter=,&xField=stop_lon&yField=stop_lat&crs=EPSG:4326'
    v = QgsVectorLayer(uri, "gtfs_stops_4326", "delimitedtext")
    if v.isValid(): stop_layers.append(v)

if not stop_layers:
    raise Exception("Nessun GTFS valido caricato.")

# Merge (se due), reproject, clip, save
if len(stop_layers) == 1:
    stops_4326 = stop_layers[0]
else:
    stops_4326 = pr.run("native:mergevectorlayers", {
        'LAYERS': stop_layers, 'CRS': None, 'OUTPUT': 'memory:'
    })['OUTPUT']

stops_25832 = pr.run("native:reprojectlayer", {
    "INPUT": stops_4326, "TARGET_CRS": "EPSG:25832", "OPERATION": "", "OUTPUT": "memory:"
})["OUTPUT"]

clip_stops = pr.run("native:clip", {
    'INPUT': stops_25832, 'OVERLAY': trento_25832, 'OUTPUT': 'memory:'
})['OUTPUT']

save_gpkg(clip_stops, GPKG_OUT, "fermate_bus")

print("--- Completato ---")
print("GeoPackage:", GPKG_OUT, "→ [comune_trento, strade, edifici, fermate_bus]")

# Riattiva rendering se era attivo prima
try:
    if _render_flag_reset is not None:
        iface.mapCanvas().setRenderFlag(_render_flag_reset)
except Exception:
    pass


## 4) Aggiungere i layer al progetto 

Questo script carica i layer, precedentemente salvati nel GeoPackage, nel progetto QGIS corrente.

### `QgsVectorLayer(uri, nome, provider)`
* **`uri`**: La stringa `f"{GPKG_OUT}|layername={ln}"` è la sintassi specifica per caricare un singolo layer (`layername=...`) da un file che ne può contenere molti (come un GeoPackage, `GPKG_OUT`).
* **`nome`**: Il nome che il layer avrà nel pannello Layer di QGIS (es. "strade").
* **`provider`**: `"ogr"` è il driver standard per i dati vettoriali (inclusi GPKG e Shapefile).

### `QgsProject.instance().addMapLayer(layer)`
* Questa è la funzione chiave. Prende l'oggetto `QgsVectorLayer` (creato e validato) e lo aggiunge visivamente al progetto QGIS (sia al pannello dei layer che alla mappa).

### `layer.featureCount()`
* Un metodo di utilità che restituisce il numero totale di feature (geometrie) presenti nel layer. Usato qui per conferma.

In [ ]:
# === QGIS: aggiungi i layer al progetto ===

# Importa le classi necessarie (QgsVectorLayer per caricare, QgsProject per aggiungere)
from qgis.core import QgsVectorLayer, QgsProject

# Itera su una lista di nomi di layer (che dovrebbero esistere nel GPKG)
for ln in ["comune_trento","strade","edifici","fermate_bus"]:
    
    # Costruisce l'URI per caricare un *singolo* layer da un GeoPackage
    # La sintassi è: "percorso/file.gpkg|layername=nome_del_layer"
    lyr = QgsVectorLayer(f"{GPKG_OUT}|layername={ln}", ln, "ogr")
    
    # Controlla se il layer è stato caricato correttamente
    if lyr.isValid():
        # Se valido, lo aggiunge al progetto QGIS corrente (pannello layer e mappa)
        QgsProject.instance().addMapLayer(lyr)
        # Stampa un messaggio di conferma con il numero di feature caricate
        print("Aggiunto:", ln, "→", lyr.featureCount())

## 5) Selezioni e export 

Questo script esegue una selezione spaziale (basata sull'area) e salva il risultato.

### `QgsProject.instance().mapLayersByName("nome")[0]`
* Recupera un layer caricato nel progetto QGIS cercandolo per nome (es. "edifici"). Restituisce il primo layer (`[0]`) trovato con quel nome.

### Iterazione e Calcolo Area (`getFeatures`, `geometry`, `area`)
* `ed.getFeatures()`: Avvia un'iterazione su *tutte* le feature del layer "edifici".
* `f.geometry()`: Ottiene la geometria della feature corrente.
* `g.area()`: Calcola l'area della geometria. **Importante**: Il calcolo è in metri quadrati (m²) perché il layer è nel CRS `EPSG:25832`, che usa il metro come unità di misura.
* `f.id()`: Restituisce l'ID univoco della feature, usato per la selezione.

### Selezione ed Esportazione
* `ed.selectByIds(lista_ids)`: Seleziona (evidenzia) nel progetto QGIS tutte le feature i cui ID sono nella lista `big_ids`.
* `pr.run("native:saveselectedfeatures", ...)`: Algoritmo di processing che prende il layer `ed` e crea un **nuovo layer in memoria** (`'OUTPUT': 'memory:'`) contenente *solo* le feature selezionate al passo precedente.
* `save_gpkg(...)`: La funzione helper (definita in una cella precedente) per salvare il nuovo layer in memoria (`sel_layer`) nel file GeoPackage.

In [ ]:
# === QGIS: selezione edifici > 80 m² ed export ===
from qgis.core import QgsProject
import processing as pr

ed = QgsProject.instance().mapLayersByName("edifici")[0]

def geom_area_m(g):
    return g.area() if g else 0.0  # CRS 25832

big_ids = [f.id() for f in ed.getFeatures() if geom_area_m(f.geometry()) > 80.0]
ed.selectByIds(big_ids)
print("Selezionati >", 80, "m²:", len(big_ids))

sel_layer = pr.run("native:saveselectedfeatures", {'INPUT': ed, 'OUTPUT': 'memory:'})['OUTPUT']
save_gpkg(sel_layer, GPKG_OUT, "edifici_over_80")
print("Esportato: edifici_over_80")

## 6) Processing: buffer → intersect 

Questo script esegue un'analisi di prossimità (vicinanza) per trovare gli edifici vicini alle fermate del bus.

### `QgsProject.instance().mapLayersByName(...)`
* Come prima, usato per recuperare i layer già caricati ("fermate_bus" e "edifici") dal progetto QGIS.

### `pr.run("native:buffer", ...)` 🗺️
* Questo è il primo algoritmo chiave. Crea un'area (un buffer) attorno a un layer.
* **`'INPUT': fermate`**: L'algoritmo gira attorno alle fermate.
* **`'DISTANCE': 300.0`**: Specifica un raggio di 300 metri (l'unità è il metro perché il CRS del progetto/layer è EPSG:25832).
* **`'DISSOLVE': True`**:  Invece di creare migliaia di cerchi separati (uno per ogni fermata), li "fonde" (dissolve) tutti in un unico, grande poligono che rappresenta l'area totale servita (entro 300m) dal trasporto pubblico. Questo rende l'operazione successiva molto più efficiente.
* **`'OUTPUT': 'memory:'`**: Il risultato (il poligono 'dissolto') viene creato in RAM.

### `pr.run("native:intersection", ...)` 
* Questo è il secondo algoritmo. Calcola l'intersezione geometrica tra due layer.
* **`'INPUT': edifici`**: Questo è il layer che vogliamo "tagliare".
* **`'OVERLAY': buf`**: Questo è il layer usato come "stampino" (il buffer di 300m creato al passo precedente).
* **Risultato**: L'output (`inter`) conterrà **solo** le porzioni di edifici che ricadono *dentro* l'area del buffer.

### `save_gpkg(...)`
* Salva il layer finale (`inter`) nel GeoPackage con il nome "edifici_entro_300m".

In [ ]:
# === QGIS: edifici entro 300 m dalle fermate ===
import processing as pr
from qgis.core import QgsProject

fermate = QgsProject.instance().mapLayersByName("fermate_bus")[0]
edifici = QgsProject.instance().mapLayersByName("edifici")[0]

buf = pr.run("native:buffer", {
    'INPUT': fermate, 'DISTANCE': 300.0, 'SEGMENTS': 12,
    'END_CAP_STYLE': 0, 'JOIN_STYLE': 0, 'MITER_LIMIT': 2,
    'DISSOLVE': True, 'OUTPUT': 'memory:'
})['OUTPUT']

inter = pr.run("native:intersection", {
    'INPUT': edifici, 'OVERLAY': buf, 'OUTPUT': 'memory:'
})['OUTPUT']

save_gpkg(inter, GPKG_OUT, "edifici_entro_300m")
print("Creato: edifici_entro_300m")

## 7) Editing attributi 


Questo script aggiunge e calcola un nuovo campo ("len_m") per il layer "strade", contenente la lunghezza di ogni segmento stradale.

### 1. Sessione di Modifica (`with edit(lyr):`)
* `with edit(lyr):` è un *context manager* (una funzione helper comune, non mostrata nel codice ma presunta).
* **Cosa fa**: Avvia automaticamente una sessione di modifica (`lyr.startEditing()`) all'inizio del blocco. Se tutto va a buon fine, **salva le modifiche** (`lyr.commitChanges()`) alla fine. Se si verifica un errore, annulla le modifiche (`lyr.rollBack()`). È il modo più **sicuro e robusto** per modificare i dati di un layer.

### 2. Aggiunta di un Campo
* `lyr.fields().indexFromName("len_m") == -1`: Questo è un controllo per vedere se il campo "len_m" **esiste già**. `indexFromName` restituisce `-1` solo se il campo non viene trovato.
* `lyr.addAttribute(QgsField(...))`: Se il campo non esiste, questa funzione lo aggiunge (crea una nuova colonna).
* `QgsField('len_m', QVariant.Double, ..., prec=2)`: Definisce la struttura del nuovo campo:
    * Nome: `len_m`
    * Tipo: `QVariant.Double` (un numero con decimali, o "float").
    * Precisione: `prec=2` (due cifre decimali).

### 3. Calcolo e Aggiornamento dei Valori
* `for f in lyr.getFeatures():`: Itera su *ogni singola feature* (strada) nel layer.
* `g.length()`: Calcola la lunghezza della geometria (`g`). Poiché il layer è in `EPSG:25832` (un CRS che usa il metro), il risultato è **direttamente in metri**.
* `f['len_m'] = ...`: Assegna il valore calcolato al campo "len_m" della feature `f` (ancora solo in memoria).
* `lyr.updateFeature(f)`: Registra la modifica per questa specifica feature nel *buffer di modifica* del layer. Le modifiche vengono poi scritte su file tutte insieme quando il blocco `with` si conclude.

In [ ]:
# === QGIS: 'len_m' sulle strade ===
from qgis.core import QgsProject, QgsField
from qgis.PyQt.QtCore import QVariant

lyr = QgsProject.instance().mapLayersByName("strade")[0]

with edit(lyr):
    if lyr.fields().indexFromName("len_m") == -1:
        lyr.addAttribute(QgsField('len_m', QVariant.Double, len=20, prec=2))
    for f in lyr.getFeatures():
        g = f.geometry()
        if g and not g.isEmpty():
            f['len_m'] = g.length()
            lyr.updateFeature(f)

print("Campo 'len_m' aggiornato.")

## 8) Statistiche per categoria 

Questo script esegue un'analisi statistica (un'aggregazione) sul layer delle strade per calcolare la lunghezza totale per ogni categoria di strada.

### `pr.run("native:aggregate", ...)`

Questo è l'algoritmo chiave ed è l'equivalente di una **`GROUP BY`.

* **`'INPUT': strade`**: Il layer di partenza con tutte le singole strade.
* **`'GROUP_BY': '"fclass"'`**: Questo è il parametro più importante. Dice a QGIS di **raggruppare** tutte le feature (strade) che hanno lo *stesso valore* nel campo `fclass`. Ad esempio, tutte le "motorway" finiscono in un gruppo, tutte le "residential" in un altro, e così via.
* **`'AGGREGATES': [...]`**: Questo definisce *quale calcolo* fare per ogni gruppo.
    * `'aggregate': 'sum'`: L'operazione da eseguire (in questo caso, una **somma**).
    * `'input': '"len_m"'`: Il campo su cui eseguire l'operazione (somma i valori del campo `len_m`).
    * `'name': 'sum_len_m'`: Il nome del nuovo campo che conterrà il risultato della somma.
* **Risultato (`'OUTPUT': 'memory:'`)**: L'output **non** è il layer delle strade. È un nuovo layer (in questo caso, una tabella senza geometria) creato in memoria, dove ogni riga rappresenta un valore univoco di `fclass` e la colonna `sum_len_m` mostra la lunghezza totale per quella categoria.


* **`save_gpkg(...)`**: Salva questa nuova tabella di statistiche nel GeoPackage.

In [ ]:
import processing as pr
from qgis.core import QgsProject

# Recupera il layer "strade" (che contiene i campi 'fclass' e 'len_m')
strade = QgsProject.instance().mapLayersByName("strade")[0]

# Esegue l'algoritmo "Aggregate" (Aggrega)
agg = pr.run("native:aggregate", {
    'INPUT': strade,        # Layer di input da analizzare
    
    # Raggruppa le feature per il campo 'fclass' 
    # (es. tutte le 'motorway' insieme, tutte le 'residential' insieme, ecc.)
    'GROUP_BY': '"fclass"', 
    
    # Definisce le operazioni da eseguire su ciascun gruppo
    'AGGREGATES': [{
        'aggregate': 'sum',      # Operazione: SOMMA
        'delimiter': ',',
        'input': '"len_m"',    # Campo da sommare: 'len_m'
        'length': 20,           
        'name': 'sum_len_m',   # Nome del nuovo campo risultato
        'precision': 2,
        'type': 6                # Tipo di dato (Double/Float)
    }],
    
    # Crea il layer risultato (la tabella aggregata) in memoria
    'OUTPUT': 'memory:'
})['OUTPUT'] # Ottiene il layer risultato

# Salva la nuova tabella di statistiche nel GeoPackage
save_gpkg(agg, GPKG_OUT, "stats_strade_per_fclass")
print("Creato: stats_strade_per_fclass")

## 9) Automazione (Funzione Parametrica: `edifici_vicini_fermate`) 

Questo script rappresenta un passo fondamentale nell'automazione: **incapsulare una logica in una funzione riutilizzabile**.

Invece di scrivere il codice per l'analisi "Buffer + Intersezione" (visto in uno script precedente) ogni volta, lo definiamo *una sola volta* in una funzione `def`,  eseguend la stessa analisi per diverse distanze (100, 250, 500).

In [ ]:
# === QGIS: funzione parametrica edifici_vicini_fermate ===
import processing as pr
from qgis.core import QgsProject, QgsVectorLayer
from pathlib import Path

# Definisce una funzione riutilizzabile per l'analisi di prossimità
def edifici_vicini_fermate(dist_m=300, edifici_name="edifici", fermate_name="fermate_bus",
                            out_layername="edifici_vicini", gpkg_path=GPKG_OUT):
    
    # Ottiene l'istanza del progetto QGIS
    proj = QgsProject.instance()
    # Cerca i layer nel progetto usando i nomi passati come parametri
    ed_list = proj.mapLayersByName(edifici_name)
    fe_list = proj.mapLayersByName(fermate_name)
    
    # Controllo di validità: lancia un errore chiaro se i layer non sono trovati
    if not ed_list or not fe_list:
        raise Exception(f"Layer mancanti: edifici='{edifici_name}' fermate='{fermate_name}'")

    # FORZA i layer nel CRS corretto (EPSG:25832) prima dell'analisi
    # (Usa la funzione 'to_epsg' definita in uno script precedente)
    ed = to_epsg(ed_list[0], 25832)
    fe = to_epsg(fe_list[0], 25832)

    # Esegue il buffer sulle fermate usando la distanza (parametro 'dist_m')
    buf = pr.run("native:buffer", {
        'INPUT': fe, 'DISTANCE': float(dist_m), 'SEGMENTS': 12, # Usa il parametro
        'END_CAP_STYLE': 0, 'JOIN_STYLE': 0, 'MITER_LIMIT': 2,
        'DISSOLVE': True, 'OUTPUT': 'memory:'
    })['OUTPUT']

    # Esegue l'intersezione tra gli edifici (riproiettati) e il buffer
    inter = pr.run("native:intersection", {'INPUT': ed, 'OVERLAY': buf, 'OUTPUT': 'memory:'})['OUTPUT']

    # Salva il risultato nel GeoPackage (usando 'save_gpkg' definita prima)
    # Usa il nome layer di output passato come parametro
    save_gpkg(inter, gpkg_path, out_layername)

    # Carica il layer appena salvato nel progetto QGIS
    lyr = QgsVectorLayer(f"{Path(gpkg_path).as_posix()}|layername={out_layername}", out_layername, "ogr")
    if lyr.isValid(): proj.addMapLayer(lyr)
    
    # Restituisce il nome del layer creato (utile per il logging)
    return out_layername

# Esegue la funzione 3 volte, in un loop, cambiando i parametri
for d in (100, 250, 500):
    print("Scritto:", edifici_vicini_fermate(
        dist_m=d,  # Passa la distanza corrente 
        out_layername=f"edifici_{d}m" # Crea un nome dinamico (edifici_100m, ecc.)
    ))

## 10) Export cartografico da codice 


Questo è uno script avanzato che crea un **layout di stampa completo (A4 Orizzontale)** al 100% da codice, lo popola con elementi cartografici e lo esporta in PDF e PNG.

### 1. Gestione del Layout
* **`QgsProject.instance().layoutManager()`**: Ottiene il gestore dei layout del progetto.
* **`lm.removeLayout(old)`**: Rimuove un layout precedente con lo stesso nome. Questo è fondamentale per rendere lo script **rieseguibile** senza creare duplicati.
* **`QgsPrintLayout(proj)`**: Crea il nuovo oggetto layout (vuoto).
* **`page.setPageSize("A4", page.Landscape)`**: Imposta la pagina in formato A4 Orizzontale.

### 2. Aggiunta della Mappa (`QgsLayoutItemMap`)
* **`QgsLayoutItemMap(layout)`**: Crea l'elemento "mappa" (il riquadro che conterrà i layer).
* **`m.attemptMove(...)` e `m.attemptResize(...)`**: Posiziona e dimensiona il riquadro della mappa (il "frame") sulla pagina, usando unità in millimetri.

### 3. Logica Chiave: Correzione "Aspect Ratio"
Questa è la parte più importante dello script. Garantisce che l'estensione della mappa (il contenuto) **non venga distorta** per adattarsi al riquadro (il frame).

1.  **Calcola i Rapporti (Ratios)**:
    * `frame_ratio`: Il rapporto tra larghezza e altezza del *riquadro* sulla pagina (es. `240 / 160`).
    * `ext_ratio`: Il rapporto tra larghezza e altezza dei *dati* (l'extent del layer "comune_trento").
2.  **Confronta i Rapporti**:
    * **Se `ext_ratio > frame_ratio`**: I *dati* sono **più larghi** (più "panoramici") del *riquadro*. Per evitare distorsioni, dobbiamo "zoomare indietro" aggiungendo spazio vuoto (padding) sopra e sotto. La larghezza dei dati (`w`) comanda.
    * **Altrimenti (`else`)**: I *dati* sono **più alti** del *riquadro*. Dobbiamo aggiungere spazio vuoto a destra e sinistra. L'altezza dei dati (`h`) comanda.
3.  **Calcola Nuova Estensione**: Vengono calcolati una `new_w` e `new_h` che hanno lo *stesso aspect ratio* del riquadro della mappa, centrati sul centroide (`cx`, `cy`) dell'extent originale.
4.  **`pad = 0.03`**: Aggiunge un piccolo margine (3%) per evitare che i dati tocchino i bordi del riquadro.
5.  **`m.setExtent(new_ext)`**: Applica questa nuova estensione calcolata (e non distorta) all'elemento mappa.

### 4. Aggiunta di Altri Elementi
* **`QgsLayoutItemLegend`**: Aggiunge la legenda. `leg.setLinkedMap(m)` è fondamentale: collega la legenda alla mappa `m` per mostrarne i layer.
* **`QgsLayoutItemScaleBar`**: Aggiunge la barra di scala, anch'essa collegata a `m` (`sb.setLinkedMap(m)`).
* **`QgsLayoutItemLabel`**: Aggiunge etichette di testo (per titolo e sottotitolo).
* **`QgsLayoutItemPicture`**: Aggiunge un'immagine. Usato per la freccia del Nord, caricando un file SVG (`.svg`) dalle risorse interne di QGIS (`:/images/...`).

### 5. Contenuto Dinamico (Espressioni)
* **`proj.setCustomVariables(...)`**: Imposta una variabile personalizzata a livello di progetto (es. `@buffer_dist`).
* **`subtitle.setUseExpression(True)`**: Dice all'etichetta di non mostrare un testo statico, ma di **valutare un'espressione QGIS**.
* **`expr_str = "concat(...)"`**: L'espressione usa la variabile `@buffer_dist` e la data corrente (`now()`) per creare un sottotitolo dinamico.
* **Blocco `else`**: Un blocco di *fallback* per versioni di QGIS più vecchie che non supportano `setUseExpression`. Valuta manualmente l'espressione e imposta il risultato come testo statico.

### 6. Esportazione
* **`lm.addLayout(layout)`**: Aggiunge il layout finito al progetto (necessario prima di esportare).
* **`QgsLayoutExporter(layout)`**: Oggetto che gestisce l'esportazione per questo layout.
* **`exp.exportToPdf(...)` e `exp.exportToImage(...)`**: Salva il layout nei formati PDF e PNG.

In [ ]:
# === QGIS: layout 100% da codice (con fix aspect ratio) ===
from pathlib import Path
from qgis.core import (
    QgsProject, QgsPrintLayout, QgsLayoutExporter,
    QgsLayoutItemMap, QgsLayoutItemLegend, QgsLayoutItemScaleBar, QgsLayoutItemLabel,
    QgsLayoutPoint, QgsLayoutSize, QgsUnitTypes, QgsExpression, QgsExpressionContext,
    QgsExpressionContextUtils, QgsRectangle
)
from qgis.PyQt.QtGui import QFont

proj = QgsProject.instance()
lm = proj.layoutManager()
LAYOUT_NAME = "Tavola Trento (auto)"
old = next((l for l in lm.layouts() if l.name() == LAYOUT_NAME), None)
if old: lm.removeLayout(old)

layout = QgsPrintLayout(proj); layout.initializeDefaults(); layout.setName(LAYOUT_NAME)
page = layout.pageCollection().pages()[0]; page.setPageSize("A4", page.Landscape)

FRAME_W_MM, FRAME_H_MM = 240.0, 160.0
trento = proj.mapLayersByName("comune_trento")[0]

m = QgsLayoutItemMap(layout); m.setId("MAP_MAIN"); m.setFrameEnabled(True)
m.attemptMove(QgsLayoutPoint(10, 25, QgsUnitTypes.LayoutMillimeters))
m.attemptResize(QgsLayoutSize(FRAME_W_MM, FRAME_H_MM, QgsUnitTypes.LayoutMillimeters))

ext = trento.extent(); w, h = ext.width(), ext.height()
frame_ratio = FRAME_W_MM / FRAME_H_MM; ext_ratio = w / h
cx = (ext.xMinimum() + ext.xMaximum()) / 2.0; cy = (ext.yMinimum() + ext.yMaximum()) / 2.0
if ext_ratio > frame_ratio: new_h, new_w = w / frame_ratio, w
else: new_w, new_h = h * frame_ratio, h
pad = 0.03; new_w *= (1 + pad); new_h *= (1 + pad)
new_ext = QgsRectangle(cx - new_w/2.0, cy - new_h/2.0, cx + new_w/2.0, cy + new_h/2.0)
m.setExtent(new_ext); layout.addItem(m)

leg = QgsLayoutItemLegend(layout); leg.setId("LEGEND_MAIN"); leg.setTitle(""); leg.setLinkedMap(m); leg.setFrameEnabled(True)
leg.attemptMove(QgsLayoutPoint(255, 25, QgsUnitTypes.LayoutMillimeters))
leg.attemptResize(QgsLayoutSize(32, 90, QgsUnitTypes.LayoutMillimeters))
layout.addItem(leg)

sb = QgsLayoutItemScaleBar(layout); sb.setLinkedMap(m); sb.setStyle("Single Box")
sb.setUnits(QgsUnitTypes.DistanceMeters); sb.setUnitLabel("m"); sb.setNumberOfSegments(4); sb.setFont(QFont("Noto Sans", 9))
sb.attemptMove(QgsLayoutPoint(10, 190, QgsUnitTypes.LayoutMillimeters)); layout.addItem(sb)

title = QgsLayoutItemLabel(layout); title.setText("Analisi accessibilità TPL — Trento")
title.setFont(QFont("Noto Sans", 16)); title.attemptMove(QgsLayoutPoint(10, 10, QgsUnitTypes.LayoutMillimeters)); layout.addItem(title)

proj.setCustomVariables({'buffer_dist': 300})
subtitle = QgsLayoutItemLabel(layout); subtitle.setFont(QFont("Noto Sans", 10))
expr_str = "concat('Buffer: ', @buffer_dist, ' m — ', format_date(now(),'yyyy-MM-dd'))"
if hasattr(subtitle, "setUseExpression"): subtitle.setUseExpression(True); subtitle.setExpression(expr_str)
else:
    ctx = QgsExpressionContext(); ctx.appendScope(QgsExpressionContextUtils.globalScope()); ctx.appendScope(QgsExpressionContextUtils.projectScope(proj))
    from qgis.core import QgsExpression; val = QgsExpression(expr_str).evaluate(ctx)
    subtitle.setText(str(val))
subtitle.attemptMove(QgsLayoutPoint(120, 14, QgsUnitTypes.LayoutMillimeters)); layout.addItem(subtitle)

try:
    from qgis.core import QgsLayoutItemPicture
    north = QgsLayoutItemPicture(layout); north.setPicturePath(":/images/svg/north_arrows/NorthArrow_02.svg")
    north.attemptMove(QgsLayoutPoint(265, 190, QgsUnitTypes.LayoutMillimeters))
    north.attemptResize(QgsLayoutSize(20, 20, QgsUnitTypes.LayoutMillimeters)); layout.addItem(north)
except Exception: pass

lm.addLayout(layout)
out_pdf = (Path(BASE_DIR) / "tavola_trento.pdf").as_posix()
out_png = (Path(BASE_DIR) / "tavola_trento.png").as_posix()
exp = QgsLayoutExporter(layout); exp.exportToPdf(out_pdf, QgsLayoutExporter.PdfExportSettings())
exp.exportToImage(out_png, QgsLayoutExporter.ImageExportSettings())
print("Esportati:", out_pdf, "e", out_png)